In [0]:
%sql

CREATE CATALOG IF NOT EXISTS quickstart_catalog;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3015769426914217>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3015769426914217>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIENBVEFMT0cgSUYgTk9UIEVYSVNUUyBxdWlja3N0YXJ0X2NhdGFsb2c=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52

In [0]:
%sql

SHOW SCHEMAS;

databaseName default

# Create Schema / Database

In [0]:
%sql
DROP SCHEMA quickstart_schema CASCADE;
CREATE SCHEMA IF NOT EXISTS quickstart_schema;
SHOW SCHEMAS;


databaseName default quickstart_schema

# Table Types



## Internal Table

### SQL

#### Create Table

In [0]:
%sql
-- DROP TABLE quickstart_schema.user_int_sql;
CREATE TABLE IF NOT EXISTS quickstart_schema.user_int_sql(
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
)  
USING CSV;

-- STORED AS TEXT FILE ROW FORMAT DELIMITED FIELDS TERMINATE BY ','
DESCRIBE FORMATTED quickstart_schema.user_int_sql;

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null # Detailed Table Information Catalog spark_catalog Database quickstart_schema Table user_int_sql Owner root Created Time Thu Jul 04 11:27:24 UTC 2024 Last Access UNKNOWN Created By Spark 3.3.2 Type MANAGED Provider CSV Location dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_sql Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.SequenceFileInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat

#### Load Data

In [0]:
dbutils.fs.cp('dbfs:/FileStore/tables/users_001-1.csv','dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_sql')

Out[115]: True

#### Select  Table

In [0]:
%sql

SELECT * from user_int_sql LIMIT 2;


id name dob email gender country region city asset marital_status

### Data Frame API



#### Create Table and Load Data

**Observations**

`
.write.mode("overwrite").format("csv").saveAsTable("quickstart_schema.user_int_df")
`
: Will write as normal table

`
.write.mode("overwrite").saveAsTable("quickstart_schema.user_int_df")
`
: Will write as Delta table

In [0]:
%sql
DROP TABLE quickstart_schema.user_int_df;

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

custom_schema = StructType(
    [
        StructField("id", IntegerType()),
        StructField("name", StringType()),
        StructField("dob", TimestampType()),
        StructField("email", StringType()),
        StructField("gender", StringType()),
        StructField("country", StringType()),
        StructField("region", StringType()),
        StructField("city", StringType()),
        StructField("asset", IntegerType()),
        StructField("marital_status", StringType()),
    ]
)


spark.read.csv(
    path="dbfs:/FileStore/tables/users_001-1.csv", header=True, schema=custom_schema
).write.mode("overwrite").format("csv").saveAsTable("quickstart_schema.user_int_df")

In [0]:
%sql

DESCRIBE FORMATTED quickstart_schema.user_int_df;

col_name data_type comment id int null name string null dob timestamp null email string null gender string null country string null region string null city string null asset int null marital_status string null # Detailed Table Information Catalog spark_catalog Database quickstart_schema Table user_int_df Owner root Created Time Thu Jul 04 11:31:41 UTC 2024 Last Access UNKNOWN Created By Spark 3.3.2 Type MANAGED Provider csv Statistics 61138 bytes Location dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_df Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.SequenceFileInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat

In [0]:
%sql

SELECT count(*) from quickstart_schema.user_int_df;

count(1) 1000

#### Data Loading Approach 01

`
.insertInto("quickstart_schema.user_int_sql")
`
Will only load data; Never create table, where as .saveAsTable("") will Create

In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

custom_schema = StructType(
    [
        StructField("id", IntegerType()),
        StructField("name", StringType()),
        StructField("dob", TimestampType()),
        StructField("email", StringType()),
        StructField("gender", StringType()),
        StructField("country", StringType()),
        StructField("region", StringType()),
        StructField("city", StringType()),
        StructField("asset", IntegerType()),
        StructField("marital_status", StringType()),
    ]
)
spark.read.csv(path="dbfs:/FileStore/tables/users_001-1.csv",
               header=True, schema=custom_schema).write.mode("append").insertInto("quickstart_schema.user_int_df")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-3015769426914260>", line 18, in <module>
    spark.read.csv(path="dbfs:/FileStore/tables/users_001-1.csv",
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/readwriter.py", line 1447, in insertInto
    self._jwrite.insertInto(tableName)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `user_int_df` cannot be found. Verify the spelling and correctness of the schema and c

In [0]:
spark.read.csv(path="dbfs:/FileStore/tables/users_001-1.csv",
               header=True, inferSchema=True).write.mode("append").saveAsTable("quickstart_schema.user_int_df")

In [0]:
spark.read.csv(path="dbfs:/FileStore/tables/users_001-1.csv",
               header=True, inferSchema=True).write.mode("append").insertInto("quickstart_schema.user_int_sql")

In [0]:
spark.sql("select * from quickstart_schema.user_int_sql LIMIT 2").display()
spark.read.table("quickstart_schema.user_int_sql").limit(3).display(n=2)

id name dob email gender country region city asset marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single

id name dob email gender country region city asset marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single 3 Ocean Workman 2024-10-10 oceanworkman2328@ymail.com Male United States Tennessee Clarksville 139985 Married

+---+---------------+----------+--------------------+------+-------------+--------+--------------+------+--------------+
| id|           name|       dob|               email|gender|      country|  region|          city| asset|marital_status|
+---+---------------+----------+--------------------+------+-------------+--------+--------------+------+--------------+
|  1|  Heather Gibbs|2024-10-31|heathergibbs6243@...|Female|United States|Virginia|Virginia Beach|734388|       Married|
|  2|Herrod Petersen|2024-02-19|herrodpetersen@ya...|  Male|United States| Arizona|       Phoenix|113506|        Single|
+---+---------------+----------+--------------------+------+-------------+--------+--------------+------+--------------+
only showing top 2 rows



In [0]:
%sql

DROP TABLE quickstart_schema.user_int CASCADE;

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-3015769426914231>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3015769426914231>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBUQUJMRSBxdWlja3N0YXJ0X3NjaGVtYS51c2VyX2ludCBDQVNDQURF").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52    

In [0]:
%sql

CREATE TABLE IF NOT EXISTS quickstart_schema.user_int_1(
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
)  ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

DESCRIBE quickstart.user_int_sql;

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null

In [0]:
%sql
DESCRIBE FORMATTED quickstart_schema.user_int_1;

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null # Detailed Table Information Catalog spark_catalog Database quickstart_schema Table user_int_1 Owner root Created Time Thu Jul 04 09:42:02 UTC 2024 Last Access UNKNOWN Created By Spark 3.3.2 Type MANAGED Provider hive Table Properties [transient_lastDdlTime=1720086122] Location dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_1 Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.TextInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat Storage Properties [field.delim=,, serialization.format=,] Partition Provider Catalog

In [0]:
%sql

SELECT * FROM quickstart.user_int_sql;

id name dob email gender country region city asset marital_status

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_1')

Out[23]: []

In [0]:
dbutils.fs.cp('dbfs:/FileStore/tables/users_001-1.csv','dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_1')

Out[30]: True

In [0]:
%sql

SELECT * FROM quickstart_schema.user_int_1;

id name dob email gender country region city asset marital_status null name null email gender country region city null marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single 3 Ocean Workman 2024-10-10 oceanworkman2328@ymail.com Male United States Tennessee Clarksville 139985 Married 4 Xaviera Maxwell 2025-03-09 xavieramaxwell@gmail.com Transgender United States Ohio Cleveland 511409 Married 5 Bo Underwood 2024-06-30 bounderwood@ymail.com Male India Tamil Nadu Madurai 366783 Married 6 Graiden Mcleod 2023-11-09 graidenmcleod1069@ymail.com Male United States Oregon Salem 638977 Divorced 7 Chantale Nixon 2023-09-06 chantalenixon404@ymail.com Female India Andaman and Nicobar Islands Port Blair 717994 Married 8 Ashton Willis 2025-02-22 ashtonwillis@gmail.com Male India Dadra and Nagar Haveli Silvassa 483841 Married 9 Mercedes Lawrence 2024-08-15 mercedeslawrence4115@ymail.com Male India Punjab Gujranwala 598378 Single 10 Norman Patton 2024-02-25 normanpatton6042@yahoomail.com Transgender India Chhattisgarh Durg 412938 Married 11 Pamela Farmer 2023-09-13 pamelafarmer7734@gmail.com Female India Pondicherry Pondicherry 961667 Common Law 12 Jana Barr 2024-04-20 janabarr3883@yahoomail.com Female United States Oklahoma Norman 319257 Married 13 Marvin Joseph 2025-02-19 marvinjoseph@ymail.com Transgender India Tamil Nadu Chennai 458643 Divorced 14 Mona Harper 2024-10-16 monaharper@gmail.com Transgender United States Wisconsin Madison 117202 Common Law 15 Teagan Barnes 2024-10-06 teaganbarnes4501@ymail.com Male United States Oregon Portland 981452 Common Law 16 Denton Raymond 2025-01-26 dentonraymond6515@yahoomail.com Transgender United States Florida Tampa 205420 Divorced 17 Dominique Horton 2023-06-11 dominiquehorton@gmail.com Transgender India Bihar Muzaffarpur 813996 Married 18 Tatyana Rosa 2024-04-05 tatyanarosa@ymail.com Female United States Minnesota Saint Paul 471709 Single 19 Len Mooney 2023-04-30 lenmooney@yahoomail.com Transgender India Tamil Nadu Thanjavur 406887 Single 20 Dean Beard 2024-03-28 deanbeard@ymail.com Female United States Utah Provo 232038 Married 21 Doris Joyce 2023-08-31 dorisjoyce@yahoomail.com Male United States Connecticut New Haven 560988 Married 22 Blythe Bryan 2023-11-08 blythebryan@yahoomail.com Male India Jammu and Kashmir Jammu 283680 Divorced 23 Rhona Irwin 2024-12-29 rhonairwin1873@ymail.com Female United States Virginia Newport News 567229 Common Law 24 John Coleman 2024-02-25 johncoleman8249@yahoomail.com Transgender India Maharastra Kolhapur 889327 Single 25 Ray Schroeder 2024-01-29 rayschroeder@ymail.com Female United States Minnesota Duluth 751145 Married 26 Elizabeth Odom 2024-11-11 elizabethodom4270@yahoomail.com Female India Odisha Raurkela Civil Township 829881 Common Law 27 Herrod Stuart 2024-10-21 herrodstuart@gmail.com Male India Rajasthan Ganganagar 144679 Common Law 28 Brent Lyons 2024-07-18 brentlyons6747@gmail.com Male United States Virginia Norfolk 174897 Common Law 29 Caldwell Hendricks 2024-06-04 caldwellhendricks1857@ymail.com Male India Nagaland Kohima 543041 Single 30 Nathan Rosario 2024-11-19 nathanrosario423@yahoomail.com Transgender India Uttarakhand Haridwar 496076 Married 31 Luke Copeland 2024-11-30 lukecopeland4730@yahoomail.com Male United States Massachusetts Springfield 633030 Divorced 32 Jonas Mcintosh 2024-04-23 jonasmcintosh947@yahoomail.com Female United States Iowa Sioux City 683453 Married 33 Rashad Watson 2024-11-24 rashadwatson@gmail.com Female India Uttar Pradesh Moradabad 747389 Divorced 34 Ignatius Bond 2024-09-15 ignatiusbond@gmail.com Male India Jammu and Kashmir Srinagar 918021 Divorced 35 Mohammad Hickman 2025-02-09 mohammadhickman8407@yahoomail.com Female India Rajasthan Kota 400670 Common Law 36 Xandra Robertson 2024-01-30 xandrarobertson697@ymail.com Transgender India Pondicherr

In [0]:
%sql
USE quickstart_schema;
SHOW TABLES;

database tableName isTemporary quickstart_schema user_int false quickstart_schema user_int_sql false

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_1')

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3015769426914232>:1
----> 1 dbutils.fs.ls('dbfs:/user/hive/warehouse/quickstart_schema.db/user_int_1')

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o586.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/quickstart_schema.db/user_int_1
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.DatabricksFile

In [0]:
%sql
DROP TABLE user_int_1;

## External Table

### SQL

#### Create Table

In [0]:
%sql

DROP TABLE `quickstart_schema`.user_ext_sql;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3015769426914280>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3015769426914280>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBUQUJMRSBxdWlja3N0YXJ0X3NjaGVtYS51c2VyX2V4dF9zcWw=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     ret

In [0]:
%sql

CREATE EXTERNAL TABLE IF NOT EXISTS quickstart_schema.user_ext_sql(
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
)  USING CSV
LOCATION 'dbfs:/FileStore/tables/user_ext_sql';



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3015769426914271>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3015769426914271>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIEVYVEVSTkFMIFRBQkxFIElGIE5PVCBFWElTVFMgcXVpY2tzdGFydF9zY2hlbWEudXNlcl9leHRfc3FsKAogIGlkIElOVCwKICBuYW1lIFNUUklORywKICBkb2IgREFURSwKICBlbWFpbCBTVFJJTkcsCiAgZ2VuZGVyIFNUUklORywKICBjb3VudHJ5IFNUUklORywKICByZWdpb24gU1RSSU5HLAogIGNpdHkgU1RSSU5HLAogIGFzc2V0IElOVCwKICBtYXJpdGFsX3N0YXR1cyBTVFJJTkcKKSAgVVNJTkcgQ1NW").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*a

#### Load Data

In [0]:
dbutils.fs.cp('dbfs:/FileStore/tables/users_001-1.csv','dbfs:/user/hive/warehouse/quickstart_schema.db/user_ext_sql')

Out[112]: True

In [0]:
%sql

SELECT * from quickstart_schema.user_ext_sql LIMIT 2;

id name dob email gender country region city asset marital_status null name null email gender country region city null marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married

### Data Frame API



#### Create Table and Load Data

In [0]:
spark.read.csv(
    path="dbfs:/FileStore/tables/users_001-1.csv", header=True, schema=custom_schema
).write.mode("overwrite").format("csv").option(
    "path", "dbfs:/FileStore/tables/user_ext_df"
).saveAsTable(
    "quickstart_schema.user_ext_df_1"
)

In [0]:
%sql

DESCRIBE FORMATTED quickstart_schema.user_ext_df_1;

col_name data_type comment id int null name string null dob timestamp null email string null gender string null country string null region string null city string null asset int null marital_status string null # Detailed Table Information Catalog spark_catalog Database quickstart_schema Table user_ext_df_1 Owner root Created Time Thu Jul 04 11:20:56 UTC 2024 Last Access UNKNOWN Created By Spark 3.3.2 Type EXTERNAL Provider csv Location dbfs:/FileStore/tables/user_ext_df Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.SequenceFileInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat

In [0]:
%sql
SELECT * from quickstart_schema.user_ext_df_1;

id name dob email gender country region city asset marital_status 1 Heather Gibbs 2024-10-31T00:00:00.000+0000 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19T00:00:00.000+0000 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single 3 Ocean Workman 2024-10-10T00:00:00.000+0000 oceanworkman2328@ymail.com Male United States Tennessee Clarksville 139985 Married 4 Xaviera Maxwell 2025-03-09T00:00:00.000+0000 xavieramaxwell@gmail.com Transgender United States Ohio Cleveland 511409 Married 5 Bo Underwood 2024-06-30T00:00:00.000+0000 bounderwood@ymail.com Male India Tamil Nadu Madurai 366783 Married 6 Graiden Mcleod 2023-11-09T00:00:00.000+0000 graidenmcleod1069@ymail.com Male United States Oregon Salem 638977 Divorced 7 Chantale Nixon 2023-09-06T00:00:00.000+0000 chantalenixon404@ymail.com Female India Andaman and Nicobar Islands Port Blair 717994 Married 8 Ashton Willis 2025-02-22T00:00:00.000+0000 ashtonwillis@gmail.com Male India Dadra and Nagar Haveli Silvassa 483841 Married 9 Mercedes Lawrence 2024-08-15T00:00:00.000+0000 mercedeslawrence4115@ymail.com Male India Punjab Gujranwala 598378 Single 10 Norman Patton 2024-02-25T00:00:00.000+0000 normanpatton6042@yahoomail.com Transgender India Chhattisgarh Durg 412938 Married 11 Pamela Farmer 2023-09-13T00:00:00.000+0000 pamelafarmer7734@gmail.com Female India Pondicherry Pondicherry 961667 Common Law 12 Jana Barr 2024-04-20T00:00:00.000+0000 janabarr3883@yahoomail.com Female United States Oklahoma Norman 319257 Married 13 Marvin Joseph 2025-02-19T00:00:00.000+0000 marvinjoseph@ymail.com Transgender India Tamil Nadu Chennai 458643 Divorced 14 Mona Harper 2024-10-16T00:00:00.000+0000 monaharper@gmail.com Transgender United States Wisconsin Madison 117202 Common Law 15 Teagan Barnes 2024-10-06T00:00:00.000+0000 teaganbarnes4501@ymail.com Male United States Oregon Portland 981452 Common Law 16 Denton Raymond 2025-01-26T00:00:00.000+0000 dentonraymond6515@yahoomail.com Transgender United States Florida Tampa 205420 Divorced 17 Dominique Horton 2023-06-11T00:00:00.000+0000 dominiquehorton@gmail.com Transgender India Bihar Muzaffarpur 813996 Married 18 Tatyana Rosa 2024-04-05T00:00:00.000+0000 tatyanarosa@ymail.com Female United States Minnesota Saint Paul 471709 Single 19 Len Mooney 2023-04-30T00:00:00.000+0000 lenmooney@yahoomail.com Transgender India Tamil Nadu Thanjavur 406887 Single 20 Dean Beard 2024-03-28T00:00:00.000+0000 deanbeard@ymail.com Female United States Utah Provo 232038 Married 21 Doris Joyce 2023-08-31T00:00:00.000+0000 dorisjoyce@yahoomail.com Male United States Connecticut New Haven 560988 Married 22 Blythe Bryan 2023-11-08T00:00:00.000+0000 blythebryan@yahoomail.com Male India Jammu and Kashmir Jammu 283680 Divorced 23 Rhona Irwin 2024-12-29T00:00:00.000+0000 rhonairwin1873@ymail.com Female United States Virginia Newport News 567229 Common Law 24 John Coleman 2024-02-25T00:00:00.000+0000 johncoleman8249@yahoomail.com Transgender India Maharastra Kolhapur 889327 Single 25 Ray Schroeder 2024-01-29T00:00:00.000+0000 rayschroeder@ymail.com Female United States Minnesota Duluth 751145 Married 26 Elizabeth Odom 2024-11-11T00:00:00.000+0000 elizabethodom4270@yahoomail.com Female India Odisha Raurkela Civil Township 829881 Common Law 27 Herrod Stuart 2024-10-21T00:00:00.000+0000 herrodstuart@gmail.com Male India Rajasthan Ganganagar 144679 Common Law 28 Brent Lyons 2024-07-18T00:00:00.000+0000 brentlyons6747@gmail.com Male United States Virginia Norfolk 174897 Common Law 29 Caldwell Hendricks 2024-06-04T00:00:00.000+0000 caldwellhendricks1857@ymail.com Male India Nagaland Kohima 543041 Single 30 Nathan Rosario 2024-11-19T00:00:00.000+0000 nathanrosario423@yahoomail.com Transgender India Uttarakhand Haridwar 496076 Married 31 Luke Copeland 2024-11-30T00:00:00.000+0000 lukecopeland4730@yahoomail.com Male United States Massachusetts Springfield 633030 Divorced 32 Jonas Mcintosh 2024-04-23T0

#### Data Loading

-------------------------

In [0]:
%sql

CREATE EXTERNAL TABLE IF NOT EXISTS quickstart_schema.user_ext(
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
)  ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'dbfs:/FileStore/tables/user_ext';



col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null

In [0]:
%sql
DESCRIBE FORMATTED quickstart_schema.user_ext;

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null # Detailed Table Information Catalog spark_catalog Database quickstart_schema Table user_ext Owner root Created Time Thu Jul 04 09:51:15 UTC 2024 Last Access UNKNOWN Created By Spark 3.3.2 Type EXTERNAL Provider hive Table Properties [transient_lastDdlTime=1720086675] Location dbfs:/FileStore/tables/user_ext Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.TextInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat Storage Properties [field.delim=,, serialization.format=,] Partition Provider Catalog

In [0]:
dbutils.fs.cp('dbfs:/FileStore/tables/users_001-1.csv','dbfs:/FileStore/tables/user_ext')

Out[45]: True

In [0]:
%sql
SELECT * from quickstart_schema.user_ext;

id name dob email gender country region city asset marital_status null name null email gender country region city null marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single 3 Ocean Workman 2024-10-10 oceanworkman2328@ymail.com Male United States Tennessee Clarksville 139985 Married 4 Xaviera Maxwell 2025-03-09 xavieramaxwell@gmail.com Transgender United States Ohio Cleveland 511409 Married 5 Bo Underwood 2024-06-30 bounderwood@ymail.com Male India Tamil Nadu Madurai 366783 Married 6 Graiden Mcleod 2023-11-09 graidenmcleod1069@ymail.com Male United States Oregon Salem 638977 Divorced 7 Chantale Nixon 2023-09-06 chantalenixon404@ymail.com Female India Andaman and Nicobar Islands Port Blair 717994 Married 8 Ashton Willis 2025-02-22 ashtonwillis@gmail.com Male India Dadra and Nagar Haveli Silvassa 483841 Married 9 Mercedes Lawrence 2024-08-15 mercedeslawrence4115@ymail.com Male India Punjab Gujranwala 598378 Single 10 Norman Patton 2024-02-25 normanpatton6042@yahoomail.com Transgender India Chhattisgarh Durg 412938 Married 11 Pamela Farmer 2023-09-13 pamelafarmer7734@gmail.com Female India Pondicherry Pondicherry 961667 Common Law 12 Jana Barr 2024-04-20 janabarr3883@yahoomail.com Female United States Oklahoma Norman 319257 Married 13 Marvin Joseph 2025-02-19 marvinjoseph@ymail.com Transgender India Tamil Nadu Chennai 458643 Divorced 14 Mona Harper 2024-10-16 monaharper@gmail.com Transgender United States Wisconsin Madison 117202 Common Law 15 Teagan Barnes 2024-10-06 teaganbarnes4501@ymail.com Male United States Oregon Portland 981452 Common Law 16 Denton Raymond 2025-01-26 dentonraymond6515@yahoomail.com Transgender United States Florida Tampa 205420 Divorced 17 Dominique Horton 2023-06-11 dominiquehorton@gmail.com Transgender India Bihar Muzaffarpur 813996 Married 18 Tatyana Rosa 2024-04-05 tatyanarosa@ymail.com Female United States Minnesota Saint Paul 471709 Single 19 Len Mooney 2023-04-30 lenmooney@yahoomail.com Transgender India Tamil Nadu Thanjavur 406887 Single 20 Dean Beard 2024-03-28 deanbeard@ymail.com Female United States Utah Provo 232038 Married 21 Doris Joyce 2023-08-31 dorisjoyce@yahoomail.com Male United States Connecticut New Haven 560988 Married 22 Blythe Bryan 2023-11-08 blythebryan@yahoomail.com Male India Jammu and Kashmir Jammu 283680 Divorced 23 Rhona Irwin 2024-12-29 rhonairwin1873@ymail.com Female United States Virginia Newport News 567229 Common Law 24 John Coleman 2024-02-25 johncoleman8249@yahoomail.com Transgender India Maharastra Kolhapur 889327 Single 25 Ray Schroeder 2024-01-29 rayschroeder@ymail.com Female United States Minnesota Duluth 751145 Married 26 Elizabeth Odom 2024-11-11 elizabethodom4270@yahoomail.com Female India Odisha Raurkela Civil Township 829881 Common Law 27 Herrod Stuart 2024-10-21 herrodstuart@gmail.com Male India Rajasthan Ganganagar 144679 Common Law 28 Brent Lyons 2024-07-18 brentlyons6747@gmail.com Male United States Virginia Norfolk 174897 Common Law 29 Caldwell Hendricks 2024-06-04 caldwellhendricks1857@ymail.com Male India Nagaland Kohima 543041 Single 30 Nathan Rosario 2024-11-19 nathanrosario423@yahoomail.com Transgender India Uttarakhand Haridwar 496076 Married 31 Luke Copeland 2024-11-30 lukecopeland4730@yahoomail.com Male United States Massachusetts Springfield 633030 Divorced 32 Jonas Mcintosh 2024-04-23 jonasmcintosh947@yahoomail.com Female United States Iowa Sioux City 683453 Married 33 Rashad Watson 2024-11-24 rashadwatson@gmail.com Female India Uttar Pradesh Moradabad 747389 Divorced 34 Ignatius Bond 2024-09-15 ignatiusbond@gmail.com Male India Jammu and Kashmir Srinagar 918021 Divorced 35 Mohammad Hickman 2025-02-09 mohammadhickman8407@yahoomail.com Female India Rajasthan Kota 400670 Common Law 36 Xandra Robertson 2024-01-30 xandrarobertson697@ymail.com Transgender India Pondicherr

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/user_ext")

Out[52]: [FileInfo(path='dbfs:/FileStore/tables/user_ext/users_001-1.csv', name='users_001-1.csv', size=54091, modificationTime=1720086769000)]

In [0]:
%sql
DROP TABLE quickstart_schema.user_ext;

In [0]:
%sql
SHOW TABLES;

database tableName isTemporary quickstart_schema user_int false quickstart_schema user_int_sql false